# Data Science Part Time Course

## Week 5 - Lesson 2 - Lab: Databases with Python

In this Lab the goal is connect to a database (created form a local file), read the data into Python and interact with it.

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [3]:
import sqlite3
conn = sqlite3.connect('../../../../data/enron.db') 
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('MessageBase',), ('RecipientBase',), ('EmployeeBase',)]

Query the `sqlite_master` table to retrieve the **schema** of the `EmployeeBase` table.

1. What fields are there?


1. What's the type of each of them?

In [5]:
fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='MessageBase';").fetchall()
print (''.join(fields[0]))

fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='RecipientBase';").fetchall()
print (''.join(fields[0]))

fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='EmployeeBase';").fetchall()
print (''.join(fields[0]))

CREATE TABLE MessageBase (
    mid INTEGER,
    filename TEXT,
    unix_time INTEGER,
    subject TEXT,
    from_eid INTEGER,
    
    PRIMARY KEY(mid ASC),
    FOREIGN KEY(from_eid) REFERENCES Employee(eid)
)
CREATE TABLE RecipientBase (
    mid INTEGER,
    rno INTEGER,
    to_eid INTEGER,
    
    PRIMARY KEY(mid ASC, rno ASC)
    FOREIGN KEY(mid) REFERENCES Message(mid)
    FOREIGN KEY(to_eid) REFERENCES Employee(eid)
)
CREATE TABLE EmployeeBase (
                  [eid] INTEGER,
  [name] TEXT,
  [department] TEXT,
  [longdepartment] TEXT,
  [title] TEXT,
  [gender] TEXT,
  [seniority] TEXT
                  
                  )


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [11]:
results = c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
for row in results:
    print (row)

(1, 'John Arnold', 'Forestry', 'ENA Gas Financial', 'VP Trading', 'Male', 'Senior')
(2, 'Harry Arora', 'Forestry', 'ENA East Power', 'VP Trading', 'Male', 'Senior')
(3, 'Robert Badeer', 'Forestry', 'ENA West Power', 'Mgr Trading', 'Male', 'Junior')
(4, 'Susan Bailey', 'Legal', 'ENA Legal', 'Specialist Legal', 'Female', 'Junior')
(5, 'Eric Bass', 'Forestry', 'ENA Gas Texas', 'Trader', 'Male', 'Junior')


In [12]:
results = c.execute("SELECT * FROM MessageBase LIMIT 5;").fetchall()
for row in results:
    print (row)

(1, 'taylor-m/sent/11', 910930020, 'Cd$ CME letter', 138)
(2, 'taylor-m/sent/17', 911459940, 'Indemnification', 138)
(3, 'taylor-m/sent/18', 911463840, 'Re: Indemnification', 138)
(4, 'taylor-m/sent/23', 911874180, 'Re: Coral Energy, L.P.', 138)
(5, 'taylor-m/sent/27', 912396120, 'Bankruptcy Code revisions', 138)


In [16]:
results = c.execute("SELECT * FROM RecipientBase LIMIT 10;").fetchall()
for row in results:
    print (row)
    
# The first field is message id, the second is recipient number, and the third is the id of the recipient.
# mid, rno, to_eid

(1, 1, 59)
(2, 1, 15)
(3, 1, 15)
(4, 1, 109)
(4, 2, 49)
(4, 3, 120)
(4, 4, 59)
(5, 1, 45)
(5, 2, 53)
(6, 1, 113)


Now try other SQL statements on the local database, such as SELECT .... FROM ... WHERE ....

For example, print the records of EmployeeBase where the Gender is male 

In [25]:
results = c.execute("SELECT * FROM EmployeeBase WHERE Gender == 'Male' LIMIT 1000;")
for row in results:
    print (row)

(1, 'John Arnold', 'Forestry', 'ENA Gas Financial', 'VP Trading', 'Male', 'Senior')
(2, 'Harry Arora', 'Forestry', 'ENA East Power', 'VP Trading', 'Male', 'Senior')
(3, 'Robert Badeer', 'Forestry', 'ENA West Power', 'Mgr Trading', 'Male', 'Junior')
(5, 'Eric Bass', 'Forestry', 'ENA Gas Texas', 'Trader', 'Male', 'Junior')
(6, 'Don Baughman Jr.', 'Forestry', 'ENA East Power', 'Mgr Trading', 'Male', 'Junior')
(8, 'Robert Benson', 'Forestry', 'ENA East Power', 'Dir Trading', 'Male', 'Senior')
(11, 'Rick Buy', 'Forestry', 'Enron', 'VP & CRO', 'Male', 'Senior')
(12, 'Larry Campbell', 'Other', 'ETS', 'Specialist', 'Male', 'Junior')
(13, 'Larry F. Campbell', 'Other', 'ENA East Power', 'Specialist', 'Male', 'Junior')
(14, 'Mike Carson', 'Forestry', 'ENA East Power', 'Mgr Trading', 'Male', 'Junior')
(18, 'Sean Crandall', 'Forestry', 'ENA West Power', 'Dir Trading', 'Male', 'Senior')
(19, 'Martin Cuilla', 'Forestry', 'ENA Gas Central', 'Mgr Trading', 'Male', 'Junior')
(20, 'Jeff Dasovich', 'Legal

#### Import each of the 3 tables to a Pandas Dataframes

In [26]:
import pandas as pd
employees = pd.read_sql("SELECT * FROM EmployeeBase;", conn)
recipients = pd.read_sql("SELECT * FROM RecipientBase;", conn)
messages = pd.read_sql("SELECT * FROM MessageBase;", conn)

In [27]:
recipients.head(10)

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49
5,4,3,120
6,4,4,59
7,5,1,45
8,5,2,53
9,6,1,113


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [28]:
employees.describe()

,eid
count,156.000000
mean,78.500000
std,45.177428
min,1.000000
25%,39.750000
50%,78.500000
75%,117.250000
max,156.000000


In [30]:
messages.describe()

,mid,unix_time,from_eid
count,21635.000000,2.163500e+04,21635.000000
mean,10818.000000,9.853194e+08,78.820800
std,6245.630873,1.972066e+07,45.203487
min,1.000000,9.109300e+08,1.000000
25%,5409.500000,9.722714e+08,42.000000
50%,10818.000000,9.870878e+08,72.000000
75%,16226.500000,1.002635e+09,120.000000
max,21635.000000,1.024667e+09,156.000000


In [31]:
recipients.describe()

,mid,rno,to_eid
count,38388.000000,38388.000000,38388.000000
mean,10952.515239,2.380900,83.140513
std,6066.838042,3.303827,44.993791
min,1.000000,1.000000,1.000000
25%,5899.000000,1.000000,47.000000
50%,11070.000000,1.000000,80.000000
75%,15964.000000,3.000000,122.000000
max,21635.000000,57.000000,156.000000


## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)